In [1]:
from client import start_client
import matplotlib.pyplot as plt
import numpy as np
import time
from matplotlib.pyplot import figure
from main_func import main,real_partial
from plotting import plot
from jupyterthemes import jtplot

client,w=start_client(9)
client

<Client: 'tcp://127.0.0.1:37685' processes=9 threads=9, memory=33.69 GB>


Client Scheduler: tcp://127.0.0.1:37685 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 9 Cores: 9 Memory: 33.69 GB


### Main function
- make_classification parameters: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html?highlight=make_classification#sklearn.datasets.make_classification
- choose e, and chunk size
- run multiple times to produce plots


### Initialize
***

In [2]:
jtplot.style(theme='grade3') #light plot theme

batches=30000 #number of minibatches
batch_size=1 #size of each minibatch
e=0.8 #tolerance for φ(χ)

#compute samples size
training_s=batches*batch_size
n_samples=int(3*training_s/2) # 1/3 of the samples will be used for testing

#parameters for make classification
dataset_params={
    "n_samples":n_samples,
    "n_features":1000,
    "n_informative":10,
    "n_redundant":0,
    "n_repeated":0,
    "n_classes":2,
    "n_clusters_per_class":1,
    "flip_y":0,
    "weights":[0.7,0.3],
    "random_state":17
}
threshold=0.03

chunks=100
e_array=[0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
sizes=[30000,50000,100000,150000,200000,300000]
minibatches=[10,50,100]
new="no" #choose if you want to recreate the dataset

### Dataset with no noise flip_y=0
***
#### Balanced
- weights=[0.5,0,5]

In [ ]:
# 3 different e
new="no"
dataset_params["weights"]=[0.5,0.5]
Acc_threshold=[]
Rounds_threshold=[]
Time_threshold=[]
labels=[[]]
l1=[[]]
l2=[[]]

for i in e_array :
    e=i
    Acc,n_rounds,time_stamps=main(client,w,new,dataset_params,e,chunks,minibatches[0])
    new="no"
    Acc_threshold.append(Acc)
    Rounds_threshold.append(n_rounds)
    Time_threshold.append(time_stamps)
    time.sleep(5)
E_real1,Acc_real1=real_partial(chunks*minibatches[0])
len_real1=np.array([i for i in range(len(Acc_real1))])

labels[0]="Real"

l1[0]=len_real1
l2[0]=Acc_real1

In [ ]:
name="Plots/Threshold_2nd_try_4x_w_"+str(len(w))+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_e_"+str(e_array)+"_a.png"
plot(l1,l2,Acc_threshold,Rounds_threshold,Time_threshold,e_array,labels,name)

-------------------------------------------------------------

Start with num of chunks: 100 ,e: 0.05
Minibatch size: 30.0
Number of chunks for each worker:  12
In progress...
coo (1, 1, 8)
Accuracy: 87.100000
coo (2, 1, 8)
Accuracy: 87.893333
coo (3, 1, 8)
Accuracy: 88.693333
coo (4, 1, 8)
Accuracy: 89.180000
coo (5, 1, 8)
Accuracy: 89.380000
coo (6, 1, 8)
Accuracy: 89.560000
coo (7, 1, 8)
Accuracy: 89.433333
coo (8, 1, 8)
Accuracy: 89.773333
coo (9, 1, 8)
Accuracy: 89.666667
coo (10, 1, 8)
Accuracy: 89.806667
coo (11, 1, 8)
Accuracy: 90.226667
coo (12, 1, 8)
Accuracy: 90.220000
coo (13, 1, 8)
Accuracy: 90.020000
coo (14, 1, 8)
Accuracy: 90.466667
coo (15, 1, 8)
Accuracy: 90.740000
coo (16, 1, 8)
Accuracy: 91.013333
coo (17, 1, 8)
Accuracy: 91.093333
coo (18, 1, 8)
Accuracy: 90.906667
coo (19, 1, 8)
Accuracy: 90.906667
coo (20, 1, 8)
Accuracy: 90.920000
coo (21, 1, 8)
Accuracy: 91.233333
coo (22, 1, 8)
Accuracy: 91.246667
coo (23, 1, 8)
Accuracy: 91.626667
coo (24, 1, 8)
Accuracy: 91.

In [ ]:
Acc_minibatch=[]
Rounds_minibatch=[]
Time_minibatch=[]
e=0.2
for i in minibatches :
    minibatch=i
    Acc,n_rounds,time_stamps=main(client,w,new,dataset_params,e,chunks,minibatch)
    new="no"
    Acc_minibatch.append(Acc)
    Rounds_minibatch.append(n_rounds)
    Time_minibatch.append(time_stamps)
    time.sleep(5)

In [ ]:
name="Plots/Minibatch__2nd_try_4x_w_"+str(len(w))+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_minib_"+str(minibatches)+"_a.png"
plot(l1,l2,Acc_minibatch,Rounds_minibatch,Time_minibatch,minibatches,labels,name)

### Save results

In [ ]:
f = open("result.txt", "a")
f.writelines(["\n\n==========================================================================="])
f.writelines(["\nNEW EXPERIMENT:\nparams:",str(dataset_params),"\nminibaches_size:"
              ,str(minibatches),"\ndifferent e's:",str(e_array),"\nn_workers: ",str(len(w))])
f.writelines(["\n___________________________________________________________________________"])
f.writelines(["\nRESULTS:\nAcc_threshold",str(Acc_threshold),"Rounds_threshold",str(Rounds_threshold),
             "Time_threshold",str(Time_threshold),"Acc_minibatch",str(Acc_minibatch),"Rounds_minibatch",str(Rounds_minibatch),
             "Time_minibatch",str(Time_minibatch),"Acc_size",str(Acc_size),
              "Rounds_size",str(Rounds_size),"Time_size",str(Time_size),
              "len_real1",str(len_real1),"Acc_real1",str(Acc_real1)])
f.close()

### Shutdown client

In [ ]:
client.close()
client.shutdown()